In [ ]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import statsmodels.tsa.api as tsa
import itertools
import statsmodels as sm
df = pd.read_csv("./train.csv")

In [ ]:
idx_list = []

del_list = []
for i in range(len(df)):
    if '12-31' not in df.loc[i, '일시']:
        del_list.append(i)

In [ ]:
df2 = df.loc[del_list, :]

df2 = df2.reset_index(drop=True)

for i in range(len(df2)):
    if (i+1)%7 == 0:
        idx_list.append(i)

df3 = df2.loc[idx_list, :]

df3

df3 = df3.reset_index(drop=True)

df3

df3[:3278-52]


df4 = df3.set_index('일시')


## 전체 모델 아리마(주간)

In [ ]:
p = range(0, 2)
d = range(0, 2)
q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 52) for x in list(itertools.product(p, d, q))]

param_list = []
param_seasonal_list = []
results_AIC_list = []

for param in pdq:
    print(param)
    for param_seasonal in seasonal_pdq:
        try:
            print(param_seasonal)
            mod = tsa.statespace.SARIMAX(df4['평균기온'][:3278-52], order=param,
                                        seasonal_order = param_seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)
            results = mod.fit()
            param_list.append(param)
            param_seasonal_list.append(param_seasonal)
            results_AIC_list.append(results.aic)
            print('param')
            print(param_list)
        except:
#             print('error')
            continue



arima_dataset = pd.DataFrame({'Parameter': param_list, 'Seasonal': param_seasonal_list, 'AIC': results_AIC_list})


In [ ]:
pd.DataFrame(arima_dataset.loc[arima_dataset['AIC'].idxmin(), :]).to_csv("arima_parameter.csv")

## 월별 아리마

In [ ]:
jan = df[df['일시'].str.contains('-01-')][['일시', '평균기온']].set_index('일시')

fev = df[df['일시'].str.contains('-02-')][['일시', '평균기온']].set_index('일시')

mar = df[df['일시'].str.contains('-03-')][['일시', '평균기온']].set_index('일시')

avr = df[df['일시'].str.contains('-04-')][['일시', '평균기온']].set_index('일시')

may = df[df['일시'].str.contains('-05-')][['일시', '평균기온']].set_index('일시')

jun = df[df['일시'].str.contains('-06-')][['일시', '평균기온']].set_index('일시')

jul = df[df['일시'].str.contains('-07-')][['일시', '평균기온']].set_index('일시')

aug = df[df['일시'].str.contains('-08-')][['일시', '평균기온']].set_index('일시')

sep = df[df['일시'].str.contains('-09-')][['일시', '평균기온']].set_index('일시')

octo = df[df['일시'].str.contains('-10-')][['일시', '평균기온']].set_index('일시')

nov = df[df['일시'].str.contains('-11-')][['일시', '평균기온']].set_index('일시')

dec = df[df['일시'].str.contains('-12-')][['일시', '평균기온']].set_index('일시')

In [ ]:
p = range(0, 2)
d = range(0, 2)
q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 31) for x in list(itertools.product(p, d, q))]

param_list_jan = []
param_seasonal_list_jan = []
results_AIC_list_jan = []

for param in pdq:
    print(param)
    for param_seasonal in seasonal_pdq:
        try:
            print(param_seasonal)
            mod_jan = tsa.statespace.SARIMAX(jan[:-31], order=param,
                                        seasonal_order = param_seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)
            results_jan = mod_jan.fit()
            param_list_jan.append(param)
            param_seasonal_list_jan.append(param_seasonal)
            results_AIC_list_jan.append(results_jan.aic)
            print('param')
            print(param_list)
        except:
#             print('error')
            continue
    
jan_dataset = pd.DataFrame({'param': param_list_jan, 'seasonal': param_seasonal_list_jan, 'aic': results_AIC_list_jan})

pd.DataFrame(jan_dataset.loc[jan_dataset['aic'].idxmin(), :])

best_mode_jan = tsa.statespace.SARIMAX(jan[:-31], order=(1, 0, 1),
                                        seasonal_order = (0, 1, 1, 31),
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)

best_results_jan = best_mode_jan.fit()
forecast_jan = best_results_jan.forecast(31)

import sklearn
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error

mae(df[df['일시'].str.contains('2022-01')]['평균기온'], forecast_jan)

p = range(0, 2)
d = range(0, 2)
q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 28) for x in list(itertools.product(p, d, q))]

param_list_fev = []
param_seasonal_list_fev = []
results_AIC_list_fev = []

for param in pdq:
    print(param)
    for param_seasonal in seasonal_pdq:
        try:
            print(param_seasonal)
            mod_fev = tsa.statespace.SARIMAX(fev[:-28], order=param,
                                        seasonal_order = param_seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)
            results_fev = mod_fev.fit()
            param_list_fev.append(param)
            param_seasonal_list_fev.append(param_seasonal)
            results_AIC_list_fev.append(results_fev.aic)
            print('param')
            print(param_list)
        except:
#             print('error')
            continue
    
fev_dataset = pd.DataFrame({'param': param_list_fev, 'seasonal': param_seasonal_list_fev, 'aic': results_AIC_list_fev})

param = pd.DataFrame(fev_dataset.loc[fev_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'param']

seasonal = pd.DataFrame(fev_dataset.loc[fev_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'seasonal']

best_mode_fev = tsa.statespace.SARIMAX(fev[:-28], order=param,
                                        seasonal_order = seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)

best_results_fev = best_mode_fev.fit()
forecast_fev = best_results_fev.forecast(28)

forecast_fev

p = range(0, 2)
d = range(0, 2)
q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 31) for x in list(itertools.product(p, d, q))]

param_list_mar = []
param_seasonal_list_mar = []
results_AIC_list_mar = []

for param in pdq:
    print(param)
    for param_seasonal in seasonal_pdq:
        try:
            print(param_seasonal)
            mod_mar = tsa.statespace.SARIMAX(mar[:-31], order=param,
                                        seasonal_order = param_seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)
            results_mar = mod_mar.fit()
            param_list_mar.append(param)
            param_seasonal_list_mar.append(param_seasonal)
            results_AIC_list_mar.append(results_mar.aic)
            print('param')
            print(param_list)
        except:
#             print('error')
            continue
    
mar_dataset = pd.DataFrame({'param': param_list_mar, 'seasonal': param_seasonal_list_mar, 'aic': results_AIC_list_mar})

param = pd.DataFrame(mar_dataset.loc[mar_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'param']

seasonal = pd.DataFrame(mar_dataset.loc[mar_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'seasonal']

best_mode_mar = tsa.statespace.SARIMAX(mar[:-31], order=param,
                                        seasonal_order = seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)

best_results_mar = best_mode_mar.fit()
forecast_mar = best_results_mar.forecast(31)

forecast_mar

p = range(0, 2)
d = range(0, 2)
q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 30) for x in list(itertools.product(p, d, q))]

param_list_avr = []
param_seasonal_list_avr = []
results_AIC_list_avr = []

for param in pdq:
    print(param)
    for param_seasonal in seasonal_pdq:
        try:
            print(param_seasonal)
            mod_avr = tsa.statespace.SARIMAX(avr[:-30], order=param,
                                        seasonal_order = param_seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)
            results_avr = mod_avr.fit()
            param_list_avr.append(param)
            param_seasonal_list_avr.append(param_seasonal)
            results_AIC_list_avr.append(results_avr.aic)
            print('param')
            print(param_list)
        except:
#             print('error')
            continue
    
avr_dataset = pd.DataFrame({'param': param_list_avr, 'seasonal': param_seasonal_list_avr, 'aic': results_AIC_list_avr})

param = pd.DataFrame(avr_dataset.loc[avr_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'param']

seasonal = pd.DataFrame(avr_dataset.loc[avr_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'seasonal']

best_mode_avr = tsa.statespace.SARIMAX(avr[:-30], order=param,
                                        seasonal_order = seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)

best_results_avr = best_mode_avr.fit()
forecast_avr = best_results_avr.forecast(30)

p = range(0, 2)
d = range(0, 2)
q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 31) for x in list(itertools.product(p, d, q))]

param_list_may = []
param_seasonal_list_may = []
results_AIC_list_may = []

for param in pdq:
    print(param)
    for param_seasonal in seasonal_pdq:
        try:
            print(param_seasonal)
            mod_may = tsa.statespace.SARIMAX(may[:-31], order=param,
                                        seasonal_order = param_seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)
            results_may = mod_may.fit()
            param_list_may.append(param)
            param_seasonal_list_may.append(param_seasonal)
            results_AIC_list_may.append(results_may.aic)
            print('param')
            print(param_list)
        except:
#             print('error')
            continue
    
may_dataset = pd.DataFrame({'param': param_list_may, 'seasonal': param_seasonal_list_may, 'aic': results_AIC_list_may})

param = pd.DataFrame(may_dataset.loc[may_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'param']

seasonal = pd.DataFrame(may_dataset.loc[may_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'seasonal']

best_mode_may = tsa.statespace.SARIMAX(may[:-31], order=param,
                                        seasonal_order = seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)

best_results_may = best_mode_may.fit()
forecast_may = best_results_may.forecast(31)

p = range(0, 2)
d = range(0, 2)
q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 30) for x in list(itertools.product(p, d, q))]

param_list_jun = []
param_seasonal_list_jun = []
results_AIC_list_jun = []

for param in pdq:
    print(param)
    for param_seasonal in seasonal_pdq:
        try:
            print(param_seasonal)
            mod_jun = tsa.statespace.SARIMAX(jun[:-30], order=param,
                                        seasonal_order = param_seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)
            results_jun = mod_jun.fit()
            param_list_jun.append(param)
            param_seasonal_list_jun.append(param_seasonal)
            results_AIC_list_jun.append(results_jun.aic)
            print('param')
            print(param_list)
        except:
#             print('error')
            continue
    
jun_dataset = pd.DataFrame({'param': param_list_jun, 'seasonal': param_seasonal_list_jun, 'aic': results_AIC_list_jun})

param = pd.DataFrame(jun_dataset.loc[jun_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'param']

seasonal = pd.DataFrame(jun_dataset.loc[jun_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'seasonal']

best_mode_jun = tsa.statespace.SARIMAX(jun[:-30], order=param,
                                        seasonal_order = seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)

best_results_jun = best_mode_jun.fit()
forecast_jun = best_results_jun.forecast(30)

p = range(0, 2)
d = range(0, 2)
q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 31) for x in list(itertools.product(p, d, q))]

param_list_jul = []
param_seasonal_list_jul = []
results_AIC_list_jul = []

for param in pdq:
    print(param)
    for param_seasonal in seasonal_pdq:
        try:
            print(param_seasonal)
            mod_jul = tsa.statespace.SARIMAX(jul[:-31], order=param,
                                        seasonal_order = param_seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)
            results_jul = mod_jul.fit()
            param_list_jul.append(param)
            param_seasonal_list_jul.append(param_seasonal)
            results_AIC_list_jul.append(results_jul.aic)
            print('param')
            print(param_list)
        except:
#             print('error')
            continue
    
jul_dataset = pd.DataFrame({'param': param_list_jul, 'seasonal': param_seasonal_list_jul, 'aic': results_AIC_list_jul})

param = pd.DataFrame(jul_dataset.loc[jul_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'param']

seasonal = pd.DataFrame(jul_dataset.loc[jul_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'seasonal']

best_mode_jul = tsa.statespace.SARIMAX(jul[:-31], order=param,
                                        seasonal_order = seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)

best_results_jul = best_mode_jul.fit()
forecast_jul = best_results_jul.forecast(31)

p = range(0, 2)
d = range(0, 2)
q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 31) for x in list(itertools.product(p, d, q))]

param_list_aug = []
param_seasonal_list_aug = []
results_AIC_list_aug = []

for param in pdq:
    print(param)
    for param_seasonal in seasonal_pdq:
        try:
            print(param_seasonal)
            mod_aug = tsa.statespace.SARIMAX(aug[:-31], order=param,
                                        seasonal_order = param_seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)
            results_aug = mod_aug.fit()
            param_list_aug.append(param)
            param_seasonal_list_aug.append(param_seasonal)
            results_AIC_list_aug.append(results_aug.aic)
            print('param')
            print(param_list)
        except:
#             print('error')
            continue
    
aug_dataset = pd.DataFrame({'param': param_list_aug, 'seasonal': param_seasonal_list_aug, 'aic': results_AIC_list_aug})

param = pd.DataFrame(aug_dataset.loc[aug_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'param']

seasonal = pd.DataFrame(aug_dataset.loc[aug_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'seasonal']

best_mode_aug = tsa.statespace.SARIMAX(aug[:-31], order=param,
                                        seasonal_order = seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)

best_results_aug = best_mode_aug.fit()
forecast_aug = best_results_aug.forecast(31)

p = range(0, 2)
d = range(0, 2)
q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 30) for x in list(itertools.product(p, d, q))]

param_list_sep = []
param_seasonal_list_sep = []
results_AIC_list_sep = []

for param in pdq:
    print(param)
    for param_seasonal in seasonal_pdq:
        try:
            print(param_seasonal)
            mod_sep = tsa.statespace.SARIMAX(sep[:-30], order=param,
                                        seasonal_order = param_seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)
            results_sep = mod_sep.fit()
            param_list_sep.append(param)
            param_seasonal_list_sep.append(param_seasonal)
            results_AIC_list_sep.append(results_sep.aic)
            print('param')
            print(param_list)
        except:
#             print('error')
            continue
    
sep_dataset = pd.DataFrame({'param': param_list_sep, 'seasonal': param_seasonal_list_sep, 'aic': results_AIC_list_sep})

param = pd.DataFrame(sep_dataset.loc[sep_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'param']

seasonal = pd.DataFrame(sep_dataset.loc[sep_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'seasonal']

best_mode_sep = tsa.statespace.SARIMAX(sep[:-30], order=param,
                                        seasonal_order = seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)

best_results_sep = best_mode_sep.fit()
forecast_sep = best_results_sep.forecast(30)

p = range(0, 2)
d = range(0, 2)
q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 31) for x in list(itertools.product(p, d, q))]

param_list_octo = []
param_seasonal_list_octo = []
results_AIC_list_octo = []

for param in pdq:
    print(param)
    for param_seasonal in seasonal_pdq:
        try:
            print(param_seasonal)
            mod_octo = tsa.statespace.SARIMAX(octo[:-31], order=param,
                                        seasonal_order = param_seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)
            results_octo = mod_octo.fit()
            param_list_octo.append(param)
            param_seasonal_list_octo.append(param_seasonal)
            results_AIC_list_octo.append(results_octo.aic)
            print('param')
            print(param_list)
        except:
#             print('error')
            continue
    
octo_dataset = pd.DataFrame({'param': param_list_octo, 'seasonal': param_seasonal_list_octo, 'aic': results_AIC_list_octo})

param = pd.DataFrame(octo_dataset.loc[octo_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'param']

seasonal = pd.DataFrame(octo_dataset.loc[octo_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'seasonal']

best_mode_octo = tsa.statespace.SARIMAX(octo[:-31], order=param,
                                        seasonal_order = seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)

best_results_octo = best_mode_octo.fit()
forecast_octo = best_results_octo.forecast(31)

p = range(0, 2)
d = range(0, 2)
q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 30) for x in list(itertools.product(p, d, q))]

param_list_nov = []
param_seasonal_list_nov = []
results_AIC_list_nov = []

for param in pdq:
    print(param)
    for param_seasonal in seasonal_pdq:
        try:
            print(param_seasonal)
            mod_nov = tsa.statespace.SARIMAX(nov[:-30], order=param,
                                        seasonal_order = param_seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)
            results_nov = mod_nov.fit()
            param_list_nov.append(param)
            param_seasonal_list_nov.append(param_seasonal)
            results_AIC_list_nov.append(results_nov.aic)
            print('param')
            print(param_list)
        except:
#             print('error')
            continue
    
nov_dataset = pd.DataFrame({'param': param_list_nov, 'seasonal': param_seasonal_list_nov, 'aic': results_AIC_list_nov})

param = pd.DataFrame(nov_dataset.loc[nov_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'param']

seasonal = pd.DataFrame(nov_dataset.loc[nov_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'seasonal']

best_mode_nov = tsa.statespace.SARIMAX(nov[:-30], order=param,
                                        seasonal_order = seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)

best_results_nov = best_mode_nov.fit()
forecast_nov = best_results_nov.forecast(30)

p = range(0, 2)
d = range(0, 2)
q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 31) for x in list(itertools.product(p, d, q))]

param_list_dec = []
param_seasonal_list_dec = []
results_AIC_list_dec = []

for param in pdq:
    print(param)
    for param_seasonal in seasonal_pdq:
        try:
            print(param_seasonal)
            mod_dec = tsa.statespace.SARIMAX(dec[:-31], order=param,
                                        seasonal_order = param_seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)
            results_dec = mod_dec.fit()
            param_list_dec.append(param)
            param_seasonal_list_dec.append(param_seasonal)
            results_AIC_list_dec.append(results_dec.aic)
            print('param')
            print(param_list)
        except:
#             print('error')
            continue
    
dec_dataset = pd.DataFrame({'param': param_list_dec, 'seasonal': param_seasonal_list_dec, 'aic': results_AIC_list_dec})

param = pd.DataFrame(dec_dataset.loc[dec_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'param']

seasonal = pd.DataFrame(dec_dataset.loc[dec_dataset['aic'].idxmin(), :]).T.reset_index().loc[0, 'seasonal']

best_mode_dec = tsa.statespace.SARIMAX(dec[:-31], order=param,
                                        seasonal_order = seasonal,
                                        enforce_stationarity = False,
                                        enforce_invertibility=False)

best_results_dec = best_mode_dec.fit()
forecast_dec = best_results_dec.forecast(31)

predict_df = pd.concat([forecast_jan.reset_index(drop=True), forecast_fev.reset_index(drop=True),
           forecast_mar.reset_index(drop=True), forecast_avr.reset_index(drop=True), 
           forecast_may.reset_index(drop=True), forecast_jun.reset_index(drop=True), 
           forecast_jul.reset_index(drop=True), forecast_aug.reset_index(drop=True), 
           forecast_sep.reset_index(drop=True), forecast_octo.reset_index(drop=True), 
           forecast_nov.reset_index(drop=True), forecast_dec.reset_index(drop=True)], axis=0).reset_index(drop=True)